In [ ]:

import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn import APPNPConv
from sklearn.metrics import roc_auc_score
import numpy as np
import torch.backends.cudnn as cudnn
import random
import pickle as pkl

In [ ]:
class APPNP(nn.Module):
    def __init__(self, in_feats, hiddens, n_classes, activation, feat_drop, edge_drop, alpha, k):
        super(APPNP, self).__init__()
        self.layers = nn.ModuleList()
        # input layer
        self.layers.append(nn.Linear(in_feats, hiddens[0]))
        # hidden layers
        for i in range(1, len(hiddens)):
            self.layers.append(nn.Linear(hiddens[i - 1], hiddens[i]))
        # output layer
        self.layers.append(nn.Linear(hiddens[-1], n_classes))
        self.activation = activation
        if feat_drop:
            self.feat_drop = nn.Dropout(feat_drop)
        else:
            self.feat_drop = lambda x: x
        self.propagate = APPNPConv(k, alpha, edge_drop)

    def reset_parameters(self):
        for layer in self.layers:
            layer.reset_parameters()

    def forward(self, g, features):
        # prediction step
        h = features
        h = self.feat_drop(h)
        h = self.activation(self.layers[0](h))
        for layer in self.layers[1:-1]:
            h = self.activation(layer(h))
        h = self.layers[-1](self.feat_drop(h))
        # propagation step
        h = self.propagate(g, h)
        return h


In [ ]:
# load G_train_dgl, G_val_dgl, G_test_dgl
# read G_train_dgl, G_val_dgl, G_test_dgl, use pickle to read
with open('G_train_dgl.gpickle', 'rb') as f:
    G_train_dgl = pkl.load(f)
    
with open('G_val_dgl.gpickle', 'rb') as f:
    G_val_dgl = pkl.load(f)
    
with open('G_test_dgl.gpickle', 'rb') as f:
    G_test_dgl = pkl.load(f)

In [ ]:
# print G_train_dgl features info
print(G_train_dgl.ndata['normalized_features'].shape)

In [ ]:
# normalize features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()


# normalize features in G_train_dgl
train_feats = G_train_dgl.ndata['features']
scaler.fit(train_feats)

In [ ]:
# print train_feats
print(train_feats)

In [ ]:
# then transform train_feats, val_feats, test_feats
train_feats_normalized = scaler.transform(train_feats)

In [ ]:
# print train_feats_normalized
print(train_feats_normalized)

In [ ]:
# do the same thing for val_feats and test_feats
val_feats = G_val_dgl.ndata['features']
val_feats_normalized = scaler.transform(val_feats)

test_feats = G_test_dgl.ndata['features']
test_feats_normalized = scaler.transform(test_feats)

In [ ]:
print (test_feats_normalized)

In [ ]:
# write back to G_train_dgl, G_val_dgl, G_test_dgl
G_train_dgl.ndata['normalized_features'] = torch.FloatTensor(train_feats_normalized)
G_val_dgl.ndata['normalized_features'] = torch.FloatTensor(val_feats_normalized)
G_test_dgl.ndata['normalized_features'] = torch.FloatTensor(test_feats_normalized)

In [ ]:
import torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score
from sklearn.metrics import f1_score, accuracy_score
import copy
import random

In [ ]:
# write ten loops to get the average AUC, F1, Precision, Recall, accuracy, macro-F1, macro-Precision, macro-Recall
for i in range(10):
    # Set the random seed, a randamly selected number
    seed = random.randint(0, 1000)
    print(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    # Create the appnp model
    model = APPNP(in_feats=8, hiddens=[128, 128], n_classes=2, activation=F.relu, feat_drop=0.1, edge_drop=0, alpha=0.5, k=3)
    # Define the optimizer and loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.BCEWithLogitsLoss()
    best_val_loss = float('inf')
    best_model = None
    num_epochs = 200
    patience = 20

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()

        labels = G_train_dgl.ndata['label'].squeeze()
        features = G_train_dgl.ndata['normalized_features']

        # Select indices of 0 and 1 labels
        zero_indices = torch.where(labels == 0)[0]
        one_indices = torch.where(labels == 1)[0]
        
        # Get the minimum count between 0 and 1 labels
        min_count = min(zero_indices.shape[0], one_indices.shape[0])
        
        # Randomly select 'min_count' indices from zero_indices and one_indices each
        selected_zero_indices = zero_indices[torch.randperm(zero_indices.shape[0])[:min_count]]
        selected_one_indices = one_indices[torch.randperm(one_indices.shape[0])[:min_count]]

        # Combine the selected indices
        selected_indices = torch.cat((selected_zero_indices, selected_one_indices))

        # Shuffle the selected indices
        selected_indices = selected_indices[torch.randperm(selected_indices.shape[0])]

        # Create a subgraph from the selected indices
        subgraph = dgl.node_subgraph(G_train_dgl, selected_indices)

        # Get the selected features and labels
        selected_features = subgraph.ndata['normalized_features']
        selected_labels = subgraph.ndata['label'].squeeze()

        # Forward pass and compute the loss
        logits = model(subgraph, selected_features)
        labels = F.one_hot(selected_labels, num_classes=2).float()
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            # Create balanced validation set
            labels = G_val_dgl.ndata['label'].squeeze()

            # Select indices of 0 and 1 labels
            zero_indices = torch.where(labels == 0)[0]
            one_indices = torch.where(labels == 1)[0]

            # Get the minimum count between 0 and 1 labels
            min_count = min(zero_indices.shape[0], one_indices.shape[0])

            # Randomly select 'min_count' indices from zero_indices and one_indices each
            selected_zero_indices = zero_indices[torch.randperm(zero_indices.shape[0])[:min_count]]
            selected_one_indices = one_indices[torch.randperm(one_indices.shape[0])[:min_count]]

            # Combine the selected indices
            selected_indices = torch.cat((selected_zero_indices, selected_one_indices))

            # Shuffle the selected indices
            selected_indices = selected_indices[torch.randperm(selected_indices.shape[0])]

            # Create a subgraph from the selected indices
            subgraph = dgl.node_subgraph(G_val_dgl, selected_indices)

            # Get the selected features and labels
            selected_features = subgraph.ndata['normalized_features']
            selected_labels = subgraph.ndata['label'].squeeze()

            # Validation
            logits = model(subgraph, selected_features)
            labels = F.one_hot(selected_labels, num_classes=2).float()
            val_loss = criterion(logits, labels)
            
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = copy.deepcopy(model)
            torch.save(model.state_dict(), 'best_model.pt')
        print(f"Epoch: {epoch + 1}/{num_epochs}, Loss: {loss:.4f}, Validation Loss: {val_loss:.4f}")

    best_model.eval()
    with torch.no_grad():
        # Create balanced testing set
        labels = G_test_dgl.ndata['label'].squeeze()

        # Select indices of 0 and 1 labels
        zero_indices = torch.where(labels == 0)[0]
        one_indices = torch.where(labels == 1)[0]

        # Get the minimum count between 0 and 1 labels
        min_count = min(zero_indices.shape[0], one_indices.shape[0])

        # Randomly select 'min_count' indices from zero_indices and one_indices each
        selected_zero_indices = zero_indices[torch.randperm(zero_indices.shape[0])[:min_count]]
        selected_one_indices = one_indices[torch.randperm(one_indices.shape[0])[:min_count]]

        # Combine the selected indices
        selected_indices = torch.cat((selected_zero_indices, selected_one_indices))

        # Shuffle the selected indices
        selected_indices = selected_indices[torch.randperm(selected_indices.shape[0])]

        # Create a subgraph from the selected indices
        subgraph = dgl.node_subgraph(G_test_dgl, selected_indices)

        # Get the selected features and labels
        selected_features = subgraph.ndata['normalized_features']
        ground_truth = subgraph.ndata['label'].squeeze()

        # Testing
        logits = best_model(subgraph, selected_features)
        _, predicted_labels = torch.max(logits, 1)

        # Calculate additional evaluation metrics for testing
        predicted_probs = F.softmax(logits, dim=1)[:, 1]
        predicted_labels = (predicted_probs > 0.5).float()
        auc = roc_auc_score(ground_truth.detach().numpy(), predicted_probs.detach().numpy())
        f1 = f1_score(ground_truth.detach().numpy(), predicted_labels.detach().numpy())
        precision = precision_score(ground_truth.detach().numpy(), predicted_labels.detach().numpy())
        recall = recall_score(ground_truth.detach().numpy(), predicted_labels.detach().numpy())
        accuracy = accuracy_score(ground_truth.detach().numpy(), predicted_labels.detach().numpy())
        macro_f1 = f1_score(ground_truth.detach().numpy(), predicted_labels.detach().numpy(), average='macro')
        macro_precision = precision_score(ground_truth.detach().numpy(), predicted_labels.detach().numpy(), average='macro')
        macro_recall = recall_score(ground_truth.detach().numpy(), predicted_labels.detach().numpy(), average='macro')
        # store results in a txt file
        with open("appnp_wo_results.txt", "a") as f:
            # need to write random seed, validation loss, test loss, auc, f1, precision, recall
            f.write(f"Random seed: {seed}, Epoch: {epoch + 1}/{num_epochs}, Loss: {loss:.4f}, Validation Loss: {val_loss:.4f}, AUC: {auc:.4f}, F1: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, Accuracy: {accuracy:.4f}, Macro-F1: {macro_f1:.4f}, Macro-Precision: {macro_precision:.4f}, Macro-recall: {macro_recall:.4f}\n")
        print(f"AUC: {auc:.4f}, F1: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, Accuracy: {accuracy:.4f}, Macro-F1: {macro_f1:.4f}, Macro-Precision: {macro_precision:.4f}, Macro-recall: {macro_recall:.4f}\n")
